# Prepare data generator

In [1]:
import pickle
from data_generator import DataGenerator

In [2]:
IMG_SIZE = (299, 299)
IMG_CHANNELS = 3
CLASSES = 42

partitions = pickle.load(open('info/partitions.bin', 'rb'))
labels = pickle.load(open('info/labels.bin', 'rb'))

# Train - freezing weights of body layers

In [3]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *

In [4]:
def new_body_freezed_model():
    base_model = keras.applications.Xception(
        include_top = False,
        input_shape = (*IMG_SIZE, IMG_CHANNELS)
    )
    
    head_model = base_model.output
    head_model = AveragePooling2D(pool_size=(7, 7))(head_model)
    head_model = Flatten(name="flatten")(head_model)
    head_model = Dense(256, activation="sigmoid")(head_model)
    head_model = Dropout(0.5)(head_model)
    head_model = Dense(CLASSES, activation="softmax")(head_model)
    
    for layer in base_model.layers:
        layer.trainable = False

    return Model(inputs=base_model.input, outputs=head_model)

In [5]:
model = new_body_freezed_model()

In [6]:
model.compile(
    optimizer = Adam(),
    loss = CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model.fit(
    DataGenerator(
        partitions['train'],
        labels,
        batch_size = 32,
        dim = IMG_SIZE,
        n_channels = IMG_CHANNELS,
        n_classes = 42,
        shuffle = True,
        dir_path = 'data/processed/train/'
    ),
    validation_data = DataGenerator(
        partitions['validation'],
        labels,
        batch_size = 1,
        dim = IMG_SIZE,
        n_channels = IMG_CHANNELS,
        n_classes = 42,
        shuffle = False,
        dir_path = 'data/processed/train/'
    ),
    epochs = 10
)

Epoch 1/10
  14/3128 [..............................] - ETA: 9:45:11 - loss: 3.8553 - accuracy: 0.0647

In [ ]:
model.save('models/xception_00_10eps')
